# python八股学习

___

## python的内存管理

### Python内存管理器
Python内存管理器负责对象内存的**分配**和**释放**，而垃圾回收器负责清理不再使用的对象。
**内存分配器**
内存分配器是python的一个多层次系统，用于管理内存的分配和释放，目的是高效分配内存减少内存碎片化。
python的内存分配器主要有几种内存分配器：
+ 大对象分配，使用系统分配器，malloc free，直接从操作系统**申请内存**，用于分配较大的内存块
+ 小对象分配,pymalloc机制：python实现的内存管理系统，管理较小内存块（小于256字节）。改分配器内部有多个内存池，大小固定且较小，多个内存池组织为一个较大的内存区域Arena。python的**私有堆**是一个存储python**所有对象**的数据结构，包括用户自定义对象和内建对象。

___

### 垃圾回收GC
python使用了**引用计数**和**循环垃圾回收**相结合的方法来实现内存管理。
+ 引用计数：每一个对象都有一个引用计数器，当对象被引用的时候，计数器会增加。引用被删除的时候，计数器会减少。当引用计数器归零的时候，该内存会被自动释放。
    + 引用计数增加行为：
        + 新建一个引用
        + 对象被作为参数传入一个函数
        + 对象被别的对象包含（列表与字典）
    + 引用计数减少行为：
        + 删除一个引用（"del"方法？）
        + 引用退出作用域
        + 对象被替换（重新赋值）

In [3]:
import sys

a = []
b = a
print(sys.getrefcount(a))  # 输出 3, 因为有3个引用（a、b 和 getrefcount 参数）
del b
print(sys.getrefcount(a))  # 输出 2


3
2


### 循环垃圾回收
当python的两个对象互相引用时，即使把他们都删除，他们的引用计数器都不能归零，因此引入**分代垃圾回收器**来处理循环引用的对象。
*假设：新创建的对象，相比于旧对象更容易成为垃圾，因为旧对象能存活到现在已经身经百战了*

循环垃圾回收的实现：标记、清除算法：
垃圾回收器从根节点（程序直接可访问的对象，包括全局变量、栈上的变量和注册的回调函数等。）对象出发，遍历由根节点直接或间接可访问的对象，并标记。扫描所有对象，对于未标记的（包含循环引用和未被标记的对象），直接清除。多次存活的对象可以晋级。
循环垃圾回收需要长期遍历所有对象，因此性能开销比较大。

分代垃圾回收器将对象分为三代，定期扫描，检测并清除循环引用：
+ G0:新创建的对象，垃圾回收器频繁扫描。
+ G1:由第一代晋升的对象，扫描频率低
+ G2:由第二代晋升的对象，扫描频率最低。

垃圾回收器在每一代中进行标记-清除操作，通过这种方式优化性能，减少垃圾回收的开销。

___


## help()与dir()函数
help是内置函数，查看函数和说明函数用途
dir也是内置函数，不带参数时，返回当前范围内的变量、方法和定义的类型列表；带参数时，返回参数的属性、方法

___


## Python程序退出的时候，是否会释放所有内存分配
不一定可以。对于python解释器可以管理的内存，比如所有对象内存和标准库分配的其他内存（文件描述符和网络连接），python可以使用引用计数和循环垃圾回收机制清空。但是对于一些c扩展和外部库，他们或许有自己的内存回收机制，这部分内存能否顺利释放需要看模块作者是否合理设计。现代的操作系统，比如windows和linux，在python进程结束后也会回收所有内存。

___


## 什么是猴子补丁
在程序运行时动态修改某个类或模块的方法或属性，不需要改动源码。灵活性高且即使生效，但是可能会导致bug、兼容性问题等。

___


## Python的数据类型：
+ 基本数据类型：int， float， bool， complex。
+ 容器类型：
  + 字符串：文本信息
  + 列表list：有序集合，可以包含任意类型的对象，列表元素可以被修改。
  + 元组tuple：与列表类似，但内容一旦创建不可修改。
  + 字典：基于键值对的数据结构，key对value，key必须是不可变类型，值可以时任意数据类型。
    + 不可变数据类型，一旦创建不可修改，即使修改，也是重新创建新对象，int float bool complex str tuple，key为不可变主要是为了保证哈希值不变。
    + 可变数据类型例如list dict之类
  + 集合set：无序、不重复的一个集合类型，只有键，没有值。
 
___


## 为什么不要轻易给python标识符以下划线开头
因为python没有定义私有变量这种东西，因此约定下划线开头的标识符代表一个私有变量。

___


## *args的含义
*args是一种函数定义中使用的参数，用于收集所有传递给函数的剩余参数，并生成一个元组。
*args必须放在所有位置参数后面，*arg后面的参数必须是关键字参数。

___

位置参数与关键字参数：

位置参数：常见且默认，输入时必须按照函数定义的顺序输入。

关键字参数：调用函数的时候指定参数值，这样可以不受顺序约束。


___

## 深拷贝与浅拷贝
**浅拷贝：** 指的是把一个对象的元素的引用拷贝到另一个对象上。这意味着，如果元对象包含了对其他对象的引用，比如列表或字典，浅拷贝就只复制这些引用，而不复制对象本身。
**深拷贝：** 深拷贝创建一个新对象，然后递归地将原对象中的元素复制到新对象中。如果原对象中的元素包括对其他对象的引用，深拷贝会复制引用的对象及其整个对象图，因此新对象和原对象将不会共享任何子对象。
**如何调用深拷贝：** 深拷贝可以使用copy模块的deepcopy()函数。

___


## 什么是python字节码
python字节码是python代码经过编译后生成的一种低级且与平台无关的代码表示方式。该字节码可以被python解释器执行，字节码使得python运行更加高效。下例使用dis模块展示python代码对应的字节码：

In [4]:
import dis

def multiply(x, y):
    return x * y

# 使用 dis.dis() 查看函数的字节码
dis.dis(multiply)


  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_MULTIPLY
              6 RETURN_VALUE


Python 在执行程序前，通常会将源代码转换成字节码（.pyc 文件）。这个过程是自动的。当你运行一个 Python 程序时，Python 解释器首先会检查是否已经存在对应的字节码版本，如果存在并且是最新的，解释器将直接执行字节码，从而跳过编译步骤，加速程序启动和运行。

___


## Cpython解释器是什么？
**Cpython**解释器是由python创始人 Guido Van Rossum 编写的python解释器。
+ 它由C语音编写，高效快速，且具有管理底层资源的能力。
+ Cpython将python代码转换为字节码，并在python虚拟机上运行
+ GIL锁：Cpython在执行字节码时使用GIL锁来确保任何时候只有一个线程在执行python字节码，简化了内存管理，但也限制了程序在多核处理器上并行执行的能力。
**其他的一些python解释器**
+ Jpython: java无锁
+ IronPython: .NET, C#
+ PyPy：有锁
+ MicroPython, 为python在微控制器和资源有限环境中使用的丐版解释器。

___


## 什么是Global Interpreter Lock？？？
官方wiki：GIL是一个互斥锁（防止两条线程同时动用进程资源，造成竞争冒险），用于保护对python对象的访问，同时防止计算一个字节码的时候的多线程操作,保护线程安全。**在GIL的作用下，python同一个进程的多个线程无法同时执行**。

python诞生于单核cpu的90年代，那是并没有考虑到多核可以同时运行多线程。这导致了即使到了多核时代，python代码也不能使用多核并行运行多线程，一次只运行一个cpu。
**Cpython的内存管理机制不是线程安全的，因此互斥锁非常有必要**
打个比方，如果三个线程都执行age=18;print(age)这两行代码，并同时执行垃圾回收线程，那么有可能垃圾回收线程会把age对象清理掉，导致一些线程无法打印出age。更严重的，导致引用计数混乱，内存泄漏。
**因此在GIL的作用下，多个线程只能分时间步运行(线程抢到gil才能运行)**
**垃圾回收回顾**
+ 引用计数
+ 标记清除
+ 分代回收

标记清除所有对象的话，非常耗费资源。因此引入分代回收机制，减少垃圾回收机制的性能开销。
**注意**
**GIL不是python的特点，是Cpython解释器的特点**
**GIL导致一个进程的多个线程无法同时运行，无法实现多核运行程序**
**GIL保证的是解释器级别的数据安全**
**写代码不需要考虑GIL，冷门知识**

## GIL的利与弊：
利：
+ 简化多线程程序编写，开发者不需要担心多线程访问对象时的线程安全问题。

弊：
性能限制，GIL限制了一次只有一个线程执行字节码。因此设计多线程密集运算程序的时候，python得不到性能提升。
但是这个问题，可以使用多进程替代多线程来解决（这样可能会占据更多资源？），或者将计算密集型运算转移到C或Cython编写的模块中。

___


## 多线程之间如何依靠GIL锁轮替?
+ 首先必须知道，在一个时间点，只有一个线程能控制解释器。
+ 在一个线程执行了一定数量的字节码或者运行一定事件后，后释放GIL锁
+ 当一个线程进入IO状态后，也会释放GIL锁。

___

## python如何实现多线程呢?
可以使用python的多线程工具包，但即使这样，也无法使用多核CPU真正进行线程并行，因为GIL锁，因此所有线程都是在cpu上轮替进行的。

___


## Python的闭包是什么?
python的闭包涉及外部函数和嵌套在内的内部函数，即使外部函数运行结束，外部函数作用域的变量依然可以由内部函数访问。闭包返回值一般是**内部函数**。
**闭包的作用：**
+ 数据隐藏： 使得某些变量变成私有变量，即，通过内部函数访问
+ 状态保持
+ 实现装饰器

In [1]:
def make_multiplier(x):
    # 外部函数定义了变量 x
    def multiplier(n):
        # 内部函数 multiplier 使用了外部函数的变量 x
        return n * x
    # 外部函数返回内部函数，这里形成了闭包
    return multiplier

# 创建一个乘以 3 的函数
times_three = make_multiplier(3)

# 使用闭包
print(times_three(10))  # 输出 30


30



___

## Python的优势是什么?
+ 易于学习
+ 完全支持面对对象编程
+ 高效的数据结构
+ 库包丰富
+ 跨平台

___


## 元组的解封装
在 Python 中，元组的解封装（tuple unpacking）是一种便捷的方式，允许你将元组中的元素直接分配到一系列变量中，而不需要通过索引访问。

In [2]:
# 定义一个元组
my_tuple = (1, 2, 3)

# 解封装元组
a, b, c = my_tuple

print(a)  # 输出 1
print(b)  # 输出 2
print(c)  # 输出 3


1
2
3


In [3]:
my_tuple = (1, 2, 3, 4, 5)
a, *b, c = my_tuple
# 带星号意思是把中间的内容赋值给一个list
print(a)  # 输出 1
print(b)  # 输出 [2, 3, 4]
print(c)  # 输出 5


1
[2, 3, 4]
5


In [4]:
my_tuple = (1, 2, 3, 4, 5)
a, _, _, _, b = my_tuple
# _代表忽略
print(a)  # 输出 1
print(b)  # 输出 5


1
5



___

## 模块
python的模块是指包含python声明和定义的文件，可以包含函数、类型、变量以及可执行代码。模块通常以.py结尾，可以被其他python代码导入和使用。
**模块的用途:**
代码重用：模块可以被多个程序或多次导入使用，避免代码重复。
命名空间隔离：模块可以帮助避免全局变量之间的冲突，因为每个模块是使用其名称空间来包含其定义的。
可维护性：模块化可以让大型程序分解成小的维护更加容易的部分。
共享服务或数据：模块可以定义函数、类和变量，这些可以被不同的程序共享使用。

**常见的内置模块与介绍：**
+ os:提供了许多操作系统服务的接口，如文件和目录访问、执行命令、处理环境变量等。
+ sys:提供对与 Python 解释器紧密相关的变量和功能的访问，例如，读写标准输入输出、操作导入的路径等。
+ math: 提供标准的算术运算函数，包括复数运算、三角函数、对数等。
+ datatime:提供处理日期和时间的类和函数
+ json:用于读取和写入 JSON 数据
+ random：随机数生成
+ collections:提供一些数据结构，比如deque， Counter

___
